In [92]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, classification_report

In [93]:
# Get twitter data files
zip_path = './new_data'  
current_directory = os.getcwd()

# Create a directory to extract files to if it doesn't exist
os.makedirs(current_directory, exist_ok=True)

# Define the paths to the CSV files
file_path = os.path.join(current_directory, './new_data/Twitter_Data.csv')

# Read the CSV files into pandas DataFrames
testing_data = pd.read_csv(file_path)

# Display the first few rows of the datasets
testing_data.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [94]:
testing_data['clean_text'] = testing_data['clean_text'].astype(str)

testing_data.dropna(subset=['clean_text'], inplace=True)

# Apply the tokenizer
sequences = tokenizer.texts_to_sequences(testing_data['clean_text'])
data_features = pad_sequences(sequences, maxlen=100)

In [95]:
# Check for any remaining non-string entries
if any(type(item) != str for item in testing_data['clean_text']):
    print("Non-string data found")

In [96]:
print(f"Original predictions: {len(predicted_labels)}")
print(f"Data rows after preprocessing: {testing_data.shape[0]}")

Original predictions: 162973
Data rows after preprocessing: 162980


In [97]:
# Remove rows where 'category' is NaN
testing_data.dropna(subset=['category'], inplace=True)

label_map = {-1.0: 'negative', 0.0: 'neutral', 1.0: 'positive'}
testing_data['category'] = testing_data['category'].map(label_map)



In [98]:
if len(predicted_labels) != testing_data.shape[0]:
    valid_indices = testing_data.index  # Indices that remained after preprocessing
    predicted_labels = [predicted_labels[i] for i in valid_indices]

In [99]:
# Encode labels
encoder = LabelEncoder()
encoder.fit(['negative', 'neutral', 'positive']) 
encoded_labels = encoder.transform(testing_data['category'])

In [100]:
# Load the pre-trained model
model = load_model('./nnmodel.h5')

# Make predictions
predictions = model.predict(data_features)
predicted_labels = predictions.argmax(axis=1)

5094/5094 [==============================] - 8s 2ms/step


In [101]:
if len(predicted_labels) != testing_data.shape[0]:
    valid_indices = testing_data.index  # Indices that remained after preprocessing
    predicted_labels = [predicted_labels[i] for i in valid_indices]

In [102]:
# If your model outputs encoded labels, decode them
predicted_labels = encoder.inverse_transform(predicted_labels)

In [104]:
# Calculate accuracy
accuracy = accuracy_score(testing_data['category'], predicted_labels)
print(f'Accuracy:, {accuracy * 100}%')

# Detailed classification report
print(classification_report(testing_data['category'], predicted_labels))

Accuracy:, 44.33249679394746%


/Users/cartermondy/Library/Python/3.8/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/cartermondy/Library/Python/3.8/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     35510
     neutral       0.00      0.00      0.00     55213
    positive       0.44      1.00      0.61     72250

    accuracy                           0.44    162973
   macro avg       0.15      0.33      0.20    162973
weighted avg       0.20      0.44      0.27    162973



/Users/cartermondy/Library/Python/3.8/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
